In [2]:
import re
import string 
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords 
import numpy as np 
import nltk
 
import gensim


In [2]:
#reading traing data 
with open(r'C:\Users\chandan sharma\Desktop\spoonsht\science_bot\science_bot\train.csv') as f:
    content = f.readlines()

In [3]:
content

['id,answerA,answerB,answerC,answerD,correctAnswer,question\n',
 "415,reflected sunlight,absorbed light from Earth's atmosphere,gases in the Moon's interior,volcanic eruptions on the Moon's surface,A,The Moon is visible to observers on Earth because of\n",
 '158,grasses -> trees -> bushes,trees -> bushes -> grasses,bushes -> grasses -> trees,grasses -> bushes -> trees,D,Which order of succession of natural communities would most likely occur in New York State?\n',
 '1959,an ion,a nucleus,a neutron,an electron,B,"If the solar system were used as a model of an atom, what would the Sun most likely represent?"\n',
 '2542,Gravity converts solid matter into gases and light energy.,Gravity causes gases and dust particles to condense into spheres.,Gravity cools gases and liquids until they become one solid mass.,Gravity pushes rocks and dust particles outward from a dense center.,B,Which of the following statements best describes the role of gravity in the formation of stars?\n',
 '1059,centim

In [4]:
#saparating column values 
train_data = [] 
for i in range(1,len(content)):
    data = {}
    x = 0  
    newline = ''  
    for j  in range(len(content[i])): 
        if(content[i][j]) == '"':     
            x = 1-x                  
        if x == 1 and content[i][j] == ',':
            continue       
        else:    
            newline += content[i][j]
                
    train = newline.split(',') 
    data['id'] = train[0]   
    data['A'] = train[1]  
    data['B'] = train[2]  
    data['C'] = train[3]   
    data['D'] = train[4]  
    data['answer'] = train[ord(train[5]) - ord('A') + 1] 
    data['res'] = train[5]
    data['question'] = train[6][:-1]   
    train_data.append(data)


In [5]:
train_data

[{'id': '415',
  'A': 'reflected sunlight',
  'B': "absorbed light from Earth's atmosphere",
  'C': "gases in the Moon's interior",
  'D': "volcanic eruptions on the Moon's surface",
  'answer': 'reflected sunlight',
  'res': 'A',
  'question': 'The Moon is visible to observers on Earth because of'},
 {'id': '158',
  'A': 'grasses -> trees -> bushes',
  'B': 'trees -> bushes -> grasses',
  'C': 'bushes -> grasses -> trees',
  'D': 'grasses -> bushes -> trees',
  'answer': 'grasses -> bushes -> trees',
  'res': 'D',
  'question': 'Which order of succession of natural communities would most likely occur in New York State?'},
 {'id': '1959',
  'A': 'an ion',
  'B': 'a nucleus',
  'C': 'a neutron',
  'D': 'an electron',
  'answer': 'a nucleus',
  'res': 'B',
  'question': '"If the solar system were used as a model of an atom what would the Sun most likely represent?"'},
 {'id': '2542',
  'A': 'Gravity converts solid matter into gases and light energy.',
  'B': 'Gravity causes gases and dus

In [6]:
#reading testdata 
with open(r'C:\Users\chandan sharma\Desktop\spoonsht\science_bot\science_bot\test.csv') as f:
    content = f.readlines()

In [7]:
#format: getting column values of test data 
test_data = [] 
for i in range(1,len(content)):
    data={}   
    flag = 0   
    newline = '' 
    for j  in range(len(content[i])): 
        if(content[i][j]) == '"':  
            flag = 1-flag         
        if flag == 1 and content[i][j] == ',':
            continue        
        else:            
            newline += content[i][j]
            
    test = newline.split(',') 
    data['id'] = test[0]   
    data['question'] = test[5][:-1] 
    data['A'] = test[1]  
    data['B'] = test[2]   
    data['C'] = test[3] 
    data['D'] = test[4]  
    test_data.append(data)

In [8]:
test_data

[{'id': '2620',
  'question': '"One type of animal hatches from an egg breathes through gills when it is young and mainly lives on land as an adult. Into which group is this animal classified?"',
  'A': 'amphibians',
  'B': 'birds',
  'C': 'mammals',
  'D': 'reptiles'},
 {'id': '2187',
  'question': 'Which is a chemical change?',
  'A': 'Element 1 is polished to form a smooth surface.',
  'B': 'Element 2 is heated and evaporates.',
  'C': '"Element 3 develops a white powdery surface after standing in air."',
  'D': 'Element 4 is separated from a mixture by filtration.'},
 {'id': '2275',
  'question': 'What type of instrument is used to record earthquakes?',
  'A': 'seismograph',
  'B': 'surface wave graph',
  'C': 'magnitude graph',
  'D': 'intensity graph'},
 {'id': '2510',
  'question': '"Which body system typically recognizes attacks and destroys foreign cells or substances that may cause disease?"',
  'A': 'digestive',
  'B': 'excretory',
  'C': 'immune',
  'D': 'respiratory'},
 {'

In [3]:
# Load Google's pre-trained Word2Vec model.
word_model = gensim.models.KeyedVectors.load_word2vec_format(r'C:\Users\chandan sharma\Desktop\spoonsht\GoogleNews-vectors-negative300.bin\GoogleNews-vectors-negative300.bin', binary=True)

MemoryError: 

In [79]:
#preprocessing of text , removing noise and stopwords. 
def preprocess(text):
    if text == "": 
        return ""  
    if type(text) is list:     
        s = ""       
        for word in text:       
            s += word + ' '  
        text = s[:-1]   
    text = text.replace("&nbsp;", ' ')  
    text = text.replace("&amp;", ' ')   
    text = text.replace('.', ' ')   
    text = text.replace(',', ' ')   
    text = text.lower()  
    text = re.sub(r'\d+', '', text) 
    exclude = set(string.punctuation) 
    for c in exclude:      
        text = text.replace(c, ' ')  
    text= ''.join(ch for ch in text if ch not in exclude) 
    text = text.strip()   
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text)  
    text = [i for i in tokens if not i in stop_words]   
    return text

In [80]:
def num(text):
    text = preprocess(text)
    x = np.ones(300)/100  
    y = np.zeros(300) 
    count = 0  
    words_not_found = 0
    for i in text:   
        try:         
            y += word_model[i] 
            count+=1      
        except KeyError:      
            words_not_found += 1  
            
            continue      
    if text == "" or count == 0:  
        return x    
    return y/count

In [81]:
x_train = []
y_train = []
 
for i in train_data: 
    #print(d['question']) 
    t = []  
    t.append(num(i['question']))
    t.append(num(i['A']))
    t.append(num(i['B'])) 
    t.append(num(i['C']))
    t.append(num(i['D'])) 
    x_train.append(t)     
    output = np.zeros(4)  
    output[ord(i['res']) - ord('A')] = 1  
    y_train.append(output)
x_train = np.array(x_train)
y_train = np.array(y_train) 
print(y_train)

[[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 ...
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]]


In [77]:
x_train.shape

(4610, 5, 300)

In [78]:
y_train.shape

(4610, 4)

In [76]:
x_test = [] 
y_test = [] 
for d in test_data:   
    t = []   
    t.append(num(d['question']))  
    t.append(num(d['A']))  
    t.append(num(d['B']))  
    t.append(num(d['C']))  
    t.append(num(d['D'])) 
    x_test.append(t)
 
 
x_test = np.array(x_test)
print(x_test.shape) 
x_test = x_test.reshape(len(x_test), 1500) 
print(x_test.shape)

(516, 5, 300)
(516, 1500)


In [50]:
x_test

array([[ 0.04426371,  0.06775309, -0.0320638 , ...,  0.27539062,
         0.09326172, -0.10302734],
       [-0.05224609,  0.06323242,  0.07324219, ..., -0.09259033,
         0.09410095, -0.04705811],
       [ 0.06074219,  0.04560547,  0.13485107, ..., -0.12792969,
        -0.15625   ,  0.04248047],
       ...,
       [ 0.01189467,  0.1051923 , -0.15538115, ...,  0.26953125,
        -0.03735352,  0.1953125 ],
       [-0.03638567,  0.12585449,  0.10710798, ..., -0.05517578,
         0.25195312, -0.07409668],
       [-0.02335612,  0.08317184,  0.10210164, ...,  0.04336548,
        -0.04510498,  0.07736206]])

In [51]:
y_test

[]

In [38]:
# reshaping data to fit in ANN 
x_train = x_train.reshape(len(x_train), 1500)

In [41]:
x_train.shape

(4610, 1500)

In [42]:
#applying LSTM model 
from keras.models import Sequential 
from keras.layers import Dense, Activation
from keras import optimizers 
from keras.regularizers import l2
 
model = Sequential([ 
    Dense(1240, input_shape=(1500,),  W_regularizer=l2(0.01)), Activation('relu'),  
    Dense(124, W_regularizer=l2(0.01)), Activation('relu'),   
    Dense(4, W_regularizer=l2(0.01)), Activation('softmax') ]) 
opt = optimizers.Adam(lr=0.0001, decay=1e-6) 
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


C:\Users\chandan sharma\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1240, input_shape=(1500,), kernel_regularizer=<keras.reg...)`
  
C:\Users\chandan sharma\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(124, kernel_regularizer=<keras.reg...)`
  if __name__ == '__main__':
C:\Users\chandan sharma\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(4, kernel_regularizer=<keras.reg...)`
  # Remove the CWD from sys.path while we load stuff.


In [44]:
#fiiting model 
model.fit(x_train, y_train, epochs=20, batch_size=64,verbose=2)

Epoch 1/20
 - 3s - loss: 1.4326 - acc: 0.4267
Epoch 2/20
 - 3s - loss: 1.4268 - acc: 0.4310
Epoch 3/20
 - 3s - loss: 1.4207 - acc: 0.4334
Epoch 4/20
 - 4s - loss: 1.4198 - acc: 0.4369
Epoch 5/20
 - 4s - loss: 1.4151 - acc: 0.4458
Epoch 6/20
 - 4s - loss: 1.4124 - acc: 0.4540
Epoch 7/20
 - 4s - loss: 1.4171 - acc: 0.4369
Epoch 8/20
 - 4s - loss: 1.4133 - acc: 0.4258
Epoch 9/20
 - 3s - loss: 1.4077 - acc: 0.4646
Epoch 10/20
 - 3s - loss: 1.4095 - acc: 0.4573
Epoch 11/20
 - 3s - loss: 1.4103 - acc: 0.4573
Epoch 12/20
 - 3s - loss: 1.4099 - acc: 0.4427
Epoch 13/20
 - 3s - loss: 1.4081 - acc: 0.4514
Epoch 14/20
 - 3s - loss: 1.4079 - acc: 0.4523
Epoch 15/20
 - 3s - loss: 1.4050 - acc: 0.4610
Epoch 16/20
 - 3s - loss: 1.4031 - acc: 0.4692
Epoch 17/20
 - 4s - loss: 1.4044 - acc: 0.4764
Epoch 18/20
 - 3s - loss: 1.4050 - acc: 0.4629
Epoch 19/20
 - 3s - loss: 1.4037 - acc: 0.4690
Epoch 20/20
 - 3s - loss: 1.4031 - acc: 0.4651


In [45]:
Y_test = model.predict(x_test)

In [47]:
#submission file 
f = open('spoonshot_final', 'w') 
f.write('id,correctAnswer\n')
i = 0 
for d in test_data:  
    ans = str(d['id']) + ',' + chr(65 + np.argmax(Y_test[i])) + '\n' 
    f.write(ans)   
    i += 1 
f.close()